# 3.1 State and Functions

In [2]:
import ts from "typescript";
import { requireCytoscape, requireCarbon, linePlot, draw, drawMemTrace, memLayout } from "./lib/draw";
import * as introspect from "./lib/introspect";
import { _List, List, Nil, Cons, arrToList, length } from "./lib/list";
import * as list from "./lib/list";
import * as graph from "./lib/graph";

requireCytoscape();
requireCarbon();

6:24 - Cannot find module './lib/graph' or its corresponding type declarations.


## Where Were We?

Concept Roadmap:

1. **Bottom-up, i.e., building blocks of languages.** (TODAY and next 2 weeks)
    - Data-Types + Recursion (last week)
    - First-Class Functions and **References** + State (this week)
2. Top-down, i.e., using building blocks.
3. *Meta-theory.*

## Goal

1. Previously, we worked with data-types which are **values**. Recall what state and **references** are and the distinction between values and references.
2. Understand the interaction between state and functions. Next time, we will see the interaction between state and first-class functions. 

## Outline

- Why state? 
- Values vs. references
- Why not state?: state and **pure functions**

## Why State?

- Performance: sometimes writing programs without state is too slow.
- Tradeoff: it is harder to reason about what our code is doing. This increases the chance to introduce bugs.
- One strategy might be write a program without state first. If it is too slow, then you can write a version that does use state.

### Example 1

- Suppose you want to concatenate some arrays together.

In [ ]:
const arr1 = [1, 2, 3];
const arr2 = [4, 5, 6];
const arr3 = arr1.concat(arr2);
arr3

In [ ]:
// arr3 is a copy
arr1[0] = -1;
arr3

In [3]:
function mutableConcat<T>(arr1: T[], arr2: T[]): void {
    for (const x of arr2) {
        arr1.push(x);
    }
}

In [4]:
const arr1 = [1, 2, 3];
const arr2 = [4, 5, 6];

// Mutable concatenation
mutableConcat(arr1, arr2);
arr1;

[ 1, 2, 3, 4, 5, 6 ]


### Performance

In [ ]:
function testMutableConcat<T>(arr: T[], count: number): T[] {
    // Add arr count times.
    const tmp: T[] = []
    for (let i = 0; i < count; i++) {
        mutableConcat(tmp, arr);
    }
    return tmp;
}

In [ ]:
function testImmutableConcat<T>(arr: T[], count: number): T[] {
    let tmp: T[] = [];
    for (let i = 0; i < count; i++) {
        tmp = tmp.concat(arr);
    }
    return tmp;
}

In [14]:
function timeFunction(name, f) {
    console.log(`--------------------------`);
    console.log(`${name} started..`);
    const t0 = process.hrtime()
    f();
    const t1 = process.hrtime(t0);
    console.log(`${f.name} completed..`);
    console.info('Execution time (hr): %ds %dms', t1[0], t1[1] / 1000000);
    return t1[0] + t1[1] / 1000000 / 1000;
}

const count = 1000;
timeFunction("Mutable", () => testMutableConcat(Array(100).fill((x) => 0), count));
timeFunction("Immutable", () => testImmutableConcat(Array(100).fill((x) => 0), count));

13:31 - Cannot find name 'testMutableConcat'.
14:33 - Cannot find name 'testImmutableConcat'.


In [15]:
const counts = [500, 1000, 1500];
const mutableTimes = [];
for (const count of counts) {
    let arr = Array(100).fill((x) => 0);
    mutableTimes.push(timeFunction("Mutable", () => testMutableConcat(arr1, count)));
}
const immutableTimes = [];
for (const count of counts) {
    let arr = Array(100).fill((x) => 0);
    immutableTimes.push(timeFunction("Immutable", () => testImmutableConcat(arr1, count)));
}

console.log(mutableTimes);
console.log(immutableTimes);

5:23 - Cannot find name 'timeFunction'.
5:53 - Cannot find name 'testMutableConcat'.
10:25 - Cannot find name 'timeFunction'.
10:57 - Cannot find name 'testImmutableConcat'.


In [16]:
linePlot(counts, [mutableTimes, immutableTimes])

1:1 - Cannot find name 'linePlot'.
1:10 - Cannot find name 'counts'.
1:19 - Cannot find name 'mutableTimes'.
1:33 - Cannot find name 'immutableTimes'.


### Computational Complexity Detour

O(N) vs. O(N^2)

### Example 2

Consider implementing the Fibonacci sequence
\begin{align*}
F_0 & = 1 \\
F_1 & = 1 \\
F_{n} & = F_{n-1} + F_{n-2} \quad\quad \mbox{when $n \geq 2$}
\end{align*}

In [21]:
function fibonacci(n: number): number {
    if (n < 0) {
        throw Error("Positive numbers only");
    }
    
    if (n == 0) { // F_0 = 1
        return 1;
    } else if (n == 1) { // F_1 = 1
        return 1;
    } else { // F_n = F_{n-1} + F_{n-2}
        return fibonacci(n-1) + fibonacci(n-2);
    }
}

for (let i=0; i < 10; i++) {
    console.log(fibonacci(i));
}

1
1
2
3
5
8
13
21
34
55


In [13]:
timeFunction("ficonnaci", () => fibonacci(20));
timeFunction("ficonnaci", () => fibonacci(30));
timeFunction("ficonnaci", () => fibonacci(40));

1:1 - Cannot find name 'timeFunction'.
2:1 - Cannot find name 'timeFunction'.
3:1 - Cannot find name 'timeFunction'.


In [ ]:
function iterFibonacci(n: number): number {
    if (n < 0) {
        throw Error("Positive numbers only");
    }
    
    let fib0: number = 0;
    let fib1: number = 1;
    let acc: number = 1;
    for (let i=1; i < n; i++) {
        fib0 = fib1;
        fib1 = acc;
        acc = fib1 + fib0;
    }
    return acc;
}

for (let i=0; i < 10; i++) {
    console.log(iterFibonacci(i));
}

In [ ]:
timeFunction("iterFibonnaci", () => iterFibonacci(20));
timeFunction("iterFibonnaci", () => iterFibonacci(30));
timeFunction("iterFibonnaci", () => iterFibonacci(40));

In [ ]:
const ns = [20, 30, 40];
const recTimes = [];
for (const n of ns) {
    recTimes.push(timeFunction("fibonnaci", () => fibonacci(n)));
}
const iterTimes = [];
for (const n of ns) {
    iterTimes.push(timeFunction("iterFibonnaci", () => iterFibonacci(n)));
}

console.log(recTimes);
console.log(iterTimes);

In [ ]:
linePlot(counts, [recTimes, iterTimes])

### Computational Complexity Detour

O(N) vs. O(e^N)

In [12]:
function cacheFibonacci(n: number): number {
    if (n < 0) {
        throw Error("Positive numbers only");
    }
    
    let cache: { [id: number]: number } = {};
    function go(n: number): number {
        if (n in cache) { // If we already computed the result, save it in the cache
            return cache[n];
        }
        
        if (n == 0) { // F_0 = 1
            cache[n] = 1;
            return 1;
        } else if (n == 1) { // F_1 = 1
            cache[n] = 1;
            return 1;
        } else { // F_n = F_{n-1} + F_{n-2}
            cache[n-1] = go(n-1);
            cache[n-2] = go(n-2);
            return cache[n-1] + cache[n-2];
        }   
    }
    
    return go(n);
}

for (let i=0; i < 10; i++) {
    console.log(cacheFibonacci(i));
}

1
1
2
3
5
8
13
21
34
55


In [ ]:
const cacheTimes = [];
for (const n of ns) {
    cacheTimes.push(timeFunction("iterFibonnaci", () => cacheFibonacci(n)));
}

In [ ]:
linePlot(counts, [iterTimes, cacheTimes])

### Computational Complexity Detour

- O(N) iterative vs. O(N) cached recursive
- This technique is called **dynamic programming**

### Summary

- Another reason to use state is to encode a notion of **time**: this event should happen before that event.
- This happens in **concurrent** and **distributed** programming.
- We will cover this later.

## References vs. Values

- We just saw that a huge reason for using state is performance.
- Now we will look more formally at **references** which is the primary mechanism by which we can store data.

### Some Examples

In [ ]:
const f = (x, y) => {
    const ans = x + y;
    x = 3;
    y = 5;
    return ans;
};


const x = 1;
const y = 3;
console.log(f(x, y))
console.log(x)
console.log(y)

In [ ]:
let x = 1;
let y = 3;
console.log(f(x, y))
console.log(x)
console.log(y)

In [ ]:
const g = (pair) => {
    const ans = pair[0] + pair[1];
    pair[0] = 3;
    pair[1] = 5;
    return ans;
}

const pair = [1, 2];
console.log(g(pair));
console.log(pair[0]);
console.log(pair[1]);

In [ ]:
const arr = []; // The reference to [] is constant
for (const x of [1, 2, 3, 4]) {
    arr.push(x);
}

### References vs. Values more formally

- Recall a **value** includes things like literal numbers and literal strings, i.e., denoted by expressions in our language that have no computation left to run.
- A **reference** is a **location value**. You can think of a location value as a number that indexes an array, and the semantics of a location value is "dereference the appropriate location of an array".

### null vs. undefined

In [7]:
console.log(null === null)
console.log(null !== null)
console.log(undefined === undefined)
console.log(undefined !== undefined)
console.log(null === undefined)
console.log(null != undefined)

true
false
true
false
false
false


In [8]:
console.log(null == null)
console.log(null != null)
console.log(undefined == undefined)
console.log(undefined != undefined)
console.log(null == undefined)       // Wait ...
console.log(null != undefined)

true
false
true
false
true
false


In [9]:
let x: string = "hello";
let y: string = null;
let z: string = undefined;

In [10]:
let a: number = 1;
let b: number = null;
let c: number = undefined;

### Intuition

- `null` is a **location value** whose semantics is "follow where I point to"
- `undefined` is a **value** whose semantics is "read me"

In [11]:
function nullUndefinedExample() {
    let s1 = "hello";
    let s2 = null;
    let s3 = undefined;
    let s4 = [s3, null, undefined];
    return s1;
}

const res: introspect.MemoryTrace = introspect.traceMemory(nullUndefinedExample, exports);
res.func()

9:12 - Cannot find namespace 'introspect'.
9:12 - Exported variable 'res' has or is using private name 'introspect'.
9:37 - Cannot find name 'introspect'.


In [ ]:
/*
```
function nullUndefinedExample() {
    let s1 = "hello";  <- HERE
    let s2 = null;
    let s3 = undefined;
    let s4 = [s3, null, undefined];
    return s4;
}
```
*/

draw(introspect.cytoscapifyMemTrace(res.memory[0], res.refId), 800, 350, memLayout)

In [ ]:
/*
```
function nullUndefinedExample() {
    let s1 = "hello";                  
    let s2 = null;                      <- HERE
    let s3 = undefined;
    let s4 = [s3, null, undefined];
    return s4;
}
```
*/

draw(introspect.cytoscapifyMemTrace(res.memory[1], res.refId), 800, 350, memLayout)

In [ ]:
/*
```
function nullUndefinedExample() {
    let s1 = "hello";                  
    let s2 = null;                      
    let s3 = undefined;                <- HERE
    let s4 = [s3, null, undefined];
    return s4;
}
```
*/

draw(introspect.cytoscapifyMemTrace(res.memory[2], res.refId), 800, 350, memLayout)

In [ ]:
/*
```
function nullUndefinedExample() {
    let s1 = "hello";                  
    let s2 = null;                      
    let s3 = undefined;                
    let s4 = [s3, null, undefined];      <- HERE
    return s4;
}
```
*/

draw(introspect.cytoscapifyMemTrace(res.memory[3], res.refId), 800, 350, memLayout)

#### Values vs References using Strings and Arrays

- Strings are values
- Arrays are references

In [ ]:
function stringArrayExample() {
    let s1 = "hello";
    let arr1 = [s1, "world"];
    let arr2 = [arr1, s1];
    return arr2;
}

const res: introspect.MemoryTrace = introspect.traceMemory(stringArrayExample, exports);
res.func()

In [ ]:
/*
```
function stringArrayExample() {
    let s1 = "hello";            <- HERE
    let arr1 = [s1, "world"];
    let arr2 = [arr1, s1];
    return arr2;
}
```
*/

draw(introspect.cytoscapifyMemTrace(res.memory[0], res.refId))

In [ ]:
/*
```
function stringArrayExample() {
    let s1 = "hello";            
    let arr1 = [s1, "world"];     <- HERE
    let arr2 = [arr1, s1];
    return arr2;
}
```
*/

draw(introspect.cytoscapifyMemTrace(res.memory[1], res.refId))

In [ ]:
/*
```
function stringArrayExample() {
    let s1 = "hello";            
    let arr1 = [s1, "world"];     
    let arr2 = [arr1, s1];           <- HERE
    return arr2;
}
```
*/

draw(introspect.cytoscapifyMemTrace(res.memory[2], res.refId))

#### Another References vs. Values example using Strings and Arrays

In [ ]:
function stringArrayExample() {
    let s1 = "hello";
    let s2 = "world";
    let arr1 = [s1, s2];
    for (let i = 0; i < 2; i++) {
        s1 = s1 + "1";
        s2 = s2 + "2";
    }
    return s1 + s2;   
}

const res: introspect.MemoryTrace = introspect.traceMemory(stringArrayExample, exports);
res.func()

In [ ]:
/*
```
function stringArrayExample() {
    let s1 = "hello";                <- HERE
    let s2 = "world";
    let arr1 = [s1, s2];
    for (let i = 0; i < 2; i++) {
        s1 = s1 + "1";
        s2 = s2 + "2";
    }
    return s1 + s2;   
}
```
*/

draw(introspect.cytoscapifyMemTrace(res.memory[0], res.refId))

In [ ]:
/*
```
function stringArrayExample() {
    let s1 = "hello";
    let s2 = "world";               <- HERE
    let arr1 = [s1, s2, [s2]];
    for (let i = 0; i < 2; i++) {
        s1 = s1 + "1";
        s2 = s2 + "2";
    }
    return s1 + s2;   
}
```
*/

draw(introspect.cytoscapifyMemTrace(res.memory[1], res.refId))

In [ ]:
/*
```
function stringArrayExample() {
    let s1 = "hello"; 
    let s2 = "world"; 
    let arr1 = [s1, s2];             // <- HERE
    for (let i = 0; i < 2; i++) {
        s1 = s1 + "1";     
        s2 = s2 + "2";
    }
    return s1 + s2;   
}
```
*/

draw(introspect.cytoscapifyMemTrace(res.memory[2], res.refId))

In [ ]:
/*
```
function stringArrayExample() {
    let s1 = "hello"; 
    let s2 = "world"; 
    let arr1 = [s1, s2];   
    for (let i = 0; i < 2; i++) {  
        s1 = s1 + "1";                // <- HERE i = 0
        s2 = s2 + "2";
    }
    return s1 + s2;   
}
```
*/

draw(introspect.cytoscapifyMemTrace(res.memory[3], res.refId))

In [ ]:
/*
```
function stringArrayExample() {
    let s1 = "hello"; 
    let s2 = "world"; 
    let arr1 = [s1, s2];   
    for (let i = 0; i < 2; i++) {  
        s1 = s1 + "1";
        s2 = s2 + "2";              // <- HERE i = 0
    }
    return s1 + s2;   
}
```
*/

draw(introspect.cytoscapifyMemTrace(res.memory[4], res.refId))

In [ ]:
/*
```
function stringArrayExample() {
    let s1 = "hello"; 
    let s2 = "world"; 
    let arr1 = [s1, s2];   
    for (let i = 0; i < 2; i++) {  
        s1 = s1 + "1";                // <- HERE i = 1
        s2 = s2 + "2";
    }
    return s1 + s2;   
}
```
*/

draw(introspect.cytoscapifyMemTrace(res.memory[5], res.refId))

In [ ]:
/*
```
function stringArrayExample() {
    let s1 = "hello"; 
    let s2 = "world"; 
    let arr1 = [s1, s2];   
    for (let i = 0; i < 2; i++) {  
        s1 = s1 + "1";                
        s2 = s2 + "2";                    // <- HERE i = 1
    }
    return s1 + s2;   
}
```
*/

draw(introspect.cytoscapifyMemTrace(res.memory[6], res.refId))

### Summary of References vs. Values

- Each variable is a reference to either a value (e.g., string) or a reference (e.g., array)
- An array `const arr = [1, 2, 3]` can be mutated because the variable `arr` is a constant reference to a reference that can be changed.

## State and Pure Functions

Summary so far:
- One huge motivation for state is performance.
- We saw that state could be implemented with the idea of a reference: it is a value that "references" another value.
- This is a powerful concept: now we'll look at why we don't want to use this concept too much.

### Example: History of Edits?

Imagine you're building software that needs undo functionality.

In [ ]:
import * as tslab from "tslab";

function createElement(elem) {
    return `<div style="padding-top: 3px;padding-right: 3px;padding-bottom: 3px;padding-left: 3px;">${elem}</div>`
}

const red = createElement(`<div style="background-color:red;width:50px;height:50px"></div>`);
const green = createElement(`<div style="background-color:green;width:50px;height:50px"></div>`);
const blue = createElement(`<div style="background-color:blue;width:50px;height:50px"></div>`);

#### Stateful approach

In [ ]:
const acc = []

acc.push(red)
acc.push(green)
acc.push(blue)
acc.push(green)
acc.push(green)

tslab.display.html(acc.join(''))

In [ ]:
// Question: Display what happened at timestep 2?

// "Reverse computation"
const tmp = [];
while (acc.length > 2) {
    tmp.push(acc.pop());
}

// Get answer
console.log("Timestep 2");
tslab.display.html(acc.join(''))

// Restore answer
for (x in tmp) {
    acc.push(x);
}

console.log("Latest Timestep");
tslab.display.html(acc.join(''))

#### Pure Approach

In [ ]:
function purePush<T>(acc: T[], x: T): T[] {
    return acc.concat(x);
}


const trace = [];
const acc = [];
const acc1 = purePush(acc, red); trace.push(acc1);
const acc2 = purePush(acc1, green); trace.push(acc2); 
const acc3 = purePush(acc2, blue); trace.push(acc3);
const acc4 = purePush(acc3, green); trace.push(acc4);
const acc5 = purePush(acc4, green); trace.push(acc5);

In [ ]:
// Question: Display what happened at timestep 2?

console.log("Timestep 2");
tslab.display.html(trace[1].join(''));

console.log("Latest Timestep");
tslab.display.html(trace[trace.length - 1].join(''))

### Example: Side-Effects?

- **Pure function**: a side-effect free function that produces the same outputs given the same inputs.
- **Impure function**: the "function" can return different outputs for the same input.

In [ ]:
let cnt = 0;

function addCnt(x: number): number {
    // Impure function because it produces different outputs for the same input
    cnt += 1;
    return x + cnt;
}

console.log(addCnt(1)); 
console.log(addCnt(1));
console.log(addCnt(1));

In [ ]:
function strangeFibonacci(n: number): number {
    // Impure function because it produces side-effects
    if (n < 0) {
        throw Error("Positive numbers only");
    }
    
    if (n % 2 === 0) {
        console.log("How many times am I printed?");
    }
    
    if (n === 0) {
        return 1;
    } else if (n === 1) {
        return 1;
    } else {
        return strangeFibonacci(n - 1) + strangeFibonacci(n - 2);
    }
}

strangeFibonacci(5)

In [ ]:
function cacheStrangeFibonacci(n: number): number {
    // Imoure function that is not equivalent to strangeFibonacci
    if (n < 0) {
        throw Error("Positive numbers only");
    }
    
    let cache: { [id: number]: number } = {};
    function go(n: number): number {
        if (n % 2 === 0) {
            console.log("How many times am I printed?");
        }
        
        if (n in cache) {
            return cache[n];
        }
        
        if (n == 0) { // F_0 = 1
            cache[n] = 1;
            return 1;
        } else if (n == 1) { // F_1 = 1
            cache[n] = 1;
            return 1;
        } else { // F_n = F_{n-1} + F_{n-2}
            cache[n-1] = go(n-1);
            cache[n-2] = go(n-2);
            return cache[n-1] + cache[n-2];
        }   
    }
    
    return go(n);
}

cacheStrangeFibonacci(5)

## Story for Today

1. One of the primary reasons to use state is for performance.
2. State is implemented in languages with references. A reference is a value that points to a location in memeory.
3. The drawback of using state is that it makes your code harder to reason about. For example, it was harder to implement an undo operation.